# GUIDED EXERCISE 8: Input and Output

With monadic programming we are able to handle IO (and more things we are used to in imperative programming)

The basic IO actions are `getChar :: IO Char`, `putChar :: Char -> IO ()` and `return :: a -> IO a`

We call them actions because they have side effects (reading something from the keyboard or printing something to the screen) in addition to return something as any other Haskell function. The returning value is always `IO x`

In [1]:
-- Checking the types
:t getChar
:t putChar
-- Well see the meaning of this in the future
:t return 

getChar :: IO Char

putChar :: Char -> IO ()

return :: forall (m :: * -> *) a. Monad m => a -> m a

In [2]:
-- Some examples
getChar
putChar 'a'
-- Be carefull this is not an Int, but a Monad Int
return 4
:t return 4
-- This will raise an error
-- 3 + (return 4)

'd'

a

4

return 4 :: forall (m :: * -> *) a. (Monad m, Num a) => m a

 **Exercise 1.** Create a function `multiplyPut Int` that receives a number, multiplies it by 2 and puts it on IO using `putChar`. Guess the type.

In [3]:
-- First version, only the first character is printed
multiplyPut :: (Show a, Num a) => a -> IO ()
multiplyPut a = putChar $ head $ show $ a * 2

In [4]:
multiplyPut 12
multiplyPut 1.5

2

3

In [5]:
-- To be able to show all characters we need to use a do
putChars :: String -> IO ()
putChars [] = putChar '\n'
putChars (x:xs) = do
    putChar x
    putChars xs
    
multiplyPut' :: (Show a, Num a) => a -> IO ()
multiplyPut' a = putChars $ show $ a * 2

In [6]:
multiplyPut' 12
multiplyPut' 1.5

24

3.0

**Performing several I/O operations in a row.** If we want to perform several I/O operations one after the other, we need to use the `do`. This will make our function to look like an imperative program.

In [7]:
-- Function that writes all the characters of a string with - between them
-- The type is IO () as the base case is IO ()
writeChars :: String -> IO ()
writeChars [x] = putChar x
writeChars (x:xs) = do
        -- Indentation must be the same for all the do block
        putChar x
        putChar '-'
        writeChars xs

In [8]:
writeChars "hello"

h-e-l-l-o

**Exercise 2.** Create a `printVowels String` function that prints the vowels of a word separated by spaces

In [9]:
vowel :: Char -> Bool
vowel x 
    | x `elem` "aeiouAEIOU" = True
    | otherwise = False

printVowels :: String -> IO ()
printVowels [x] = if vowel x then putChar x else return ()
printVowels (x:xs) = if not (vowel x) then printVowels xs
        else do
            putChar x
            putChar ' '
            printVowels xs

Line 7: Use when
Found:
if vowel x then putChar x else return ()
Why not:
Control.Monad.when (vowel x) $ putChar x

In [10]:
printVowels "hello how are you, I am OK"

e o o a e o u I a O

The `getChar` function does not return a `Char` but an `IO Char` which is not compatible. How do we work with the `Char` in `getChar :: IO Char`? 

In [11]:
-- Function that attempts to read a char and return it
-- Not working as it uses =
-- Notice it returns a monad of IO Char (we'll see monads next week) but not the Char
wrongReadChar :: Monad m => m (IO Char)
wrongReadChar = return getChar

We need to use the `<-` operator inside a `do`. Notice the `=` does not work for `IO` as you would get the whole `IO Char`

In [1]:
-- Function that reads two chars and returns a String with them
-- Notice String is defined as type String = [Char], so we could also put IO [Char]
readTwoChars :: IO String 
readTwoChars = do 
    -- Indentation must be equal for all the do block
    -- The <- operator binds the result of an IO to a name
    x <- getChar
    -- This is to discard the /n
    getChar
    y <- getChar
    -- This is to discard the /n
    getChar
    return [x, y]

In [2]:
readTwoChars

"ff"

**Exercise 3.** Create a function `equalChars` that reads two chars and returns if they are equal or not

In [3]:
equalChars :: IO Bool
equalChars = do         
                -- We'll use the previous one
                x <- readTwoChars
                putStrLn $ "The first character is " ++ show (head x)
                putStrLn $ "The second character is " ++ show (last x)
                return (head x == last x)

In [4]:
equalChars

The first character is 'a'
The second character is 'a'
True

The `getChar` function is generalized by `getLine` (see slides). We have also `putStr` and `putStrLn`

In [5]:
-- Try it without the do, to see what happens
putStrLn "Enter you name"
do 
    x <- getLine
    putStr "hello! " 
    putStr x
    putStrLn " How are you?"

-- Types
:t getLine
:t putStr

Enter you name

hello! Pepe How are you?

getLine :: IO String

putStr :: String -> IO ()

With `getChar` and `getLine` we read a `Char` and a `String` respectively, but how do we read numbers? We can read a `String` and use `let` and the `read` method:

In [13]:
evenInput :: IO String
evenInput = do
            putStrLn "Enter a number"
            sn <- getLine
            putStrLn ("You entered " ++ sn)
            let n = read sn :: Int
            if even n then return "Even" else return "Odd"

In [14]:
evenInput

Enter a number
You entered 45
"Odd"

**Exercise 4.** Create a function that asks for two integer numbers, adds them and returns the result.

In [15]:
addNumbers :: IO ()
addNumbers = do
        putStrLn "Enter the first number"
        sn1 <- getLine
        putStrLn sn1
        putStrLn "Enter the second number"
        sn2 <- getLine
        putStrLn sn2
        let n1 = read sn1 :: Int
            n2 = read sn2 :: Int
        putStrLn (sn1 ++ " + " ++ sn2 ++ " = " ++ show (n1 + n2))

In [16]:
addNumbers

Enter the first number
12
Enter the second number
85
12 + 85 = 97

How do we get the read numbers out of the function? In other languages we are used to do something like 
 `result = readNumber()` and then work with `result`. In Haskell we cannot do it, but we can invoke another function on a read value:

In [17]:
inLimits :: Ord a => a -> a -> a -> Bool
inLimits lb up value = lb <= value && value <= up

checkLimits = do
    putStrLn "Enter the lower bound"
    lb <- getLine
    putStrLn lb
    putStrLn "Enter the upper bound"
    ub <- getLine
    putStrLn ub
    putStrLn "Enter the value to check"
    v <- getLine
    putStrLn v
    let 
        lb1 = read lb :: Int
        ub1 = read ub :: Int
        v1 = read v :: Int
    return (inLimits lb1 ub1 v1)
    

In [11]:
checkLimits

Enter the lower bound
1
Enter the upper bound
10
Enter the value to check
5
True

Check the result of the following program. As you see in Haskell `return` is not like in imperative languages, it should be read as *wrap the datum to an IO type*

In [19]:
function = do
    putStrLn "hello"
    return 4
    putStrLn "bye"

Line 3: Redundant return
Found:
return 4
Why not:

In [20]:
function

hello
bye

In [24]:
-- print is an alias for putStrLn . show
putStrLn . show $ 3
print 3

Line 2: Use print
Found:
putStrLn . show
Why not:
print

3

3